# Compare multiple string columns to return similarity scores and longest partial string matches

In [ ]:
import numpy as np
import pandas as pd
from difflib import SequenceMatcher

In [ ]:
file = 'data1.csv'
data1 = pd.read_csv(file)

In [ ]:
file = 'data2.csv'
data2 = pd.read_csv(file)

In [ ]:
frames = [data1, data2]
 
res_dt = pd.concat(frames).reset_index()
del res_dt['index']

In [ ]:
data = res_dt.replace(np.nan, '', regex=True)

In [ ]:
data.drop_duplicates(subset='name', keep='first', inplace=True)

Use SequenceMatcher and find string similarity between string columns, score is out of 100, round up (2 digits)

In [ ]:
from difflib import SequenceMatcher

def sim_score(df, col1, col2):
    return round((SequenceMatcher(None, df[col1], df[col2]).ratio()*100),2)

Compute similarity score for multiple string columns, compare and return maximum similarity score

In [ ]:
data['score'] = data.apply(sim_score,
                          args=('col1', 'col2'), axis=1)
data['score2'] = data.apply(sim_score,
                          args=('col1', 'col3'), axis=1)

data['max_score'] = data[['score1', 'score2']].max(axis=1)
data

In [ ]:
data.to_csv("match_score.csv")

Compute similarity score for multiple string columns, and find matched longest keyword (string)

In [ ]:
from difflib import SequenceMatcher

def sim_keyword(df, col1, col2):
    seqMatch=SequenceMatcher(None, df[col1], df[col2])
    match=seqMatch.find_longest_match(0, len(df[col1]), 0, len(df[col2])) 
    if (match.size!=0): 
          return (df[col1][match.a: match.a + match.size])  
    else: 
          return ''

In [ ]:
data['matched_keyword'] = data.apply(sim_keyword,
                          args=('col1', 'col2'), axis=1)
data['matched_keyword2'] = data.apply(sim_keyword,
                          args=('col1', 'col3'), axis=1)

data

In [ ]:
#remove '-' , numeric matches, and single character letter

data2=data[~data['matched_keyword'].str.contains('0|1|2|3|4|5|6|7|8|9')]
data2['matched_keyword'] = data2['matched_keyword'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ').str.replace('-', '')
data2

In [ ]:
data2.to_csv("match keyword.csv")